<a href="https://colab.research.google.com/github/JEBAKUMAR12/MACHINE-LEARNING-PROJECT/blob/main/Copy_of_ML_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, LSTM, TimeDistributed
import os

# --- Configuration ---
MODEL_PATH = "models/steering_model.h5"  # Replace with your actual model path
IMAGE_WIDTH = 200
IMAGE_HEIGHT = 66
SEQUENCE_LENGTH = 5  # Number of frames the model expects

# --- Build Model Architecture (Optional, for reference or future training) ---
def build_model():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(24, (5, 5), activation='relu', strides=(2, 2)),
                              input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(Conv2D(36, (5, 5), activation='relu', strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(48, (5, 5), activation='relu', strides=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# --- Load Trained Model ---
model = None
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print(f"✅ Model loaded successfully from {MODEL_PATH}")
else:
    print(f"❌ Model not found at {MODEL_PATH}. Exiting.")
    exit()

# --- Simulate Camera Capture ---
def capture_frame():
    dummy_image = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=np.uint8)
    cv2.line(dummy_image, (IMAGE_WIDTH // 2, 0), (IMAGE_WIDTH // 2, IMAGE_HEIGHT), (0, 255, 0), 2)
    return dummy_image

# --- Initialize Frame Sequence ---
image_sequence = []

for _ in range(SEQUENCE_LENGTH):
    frame = capture_frame()
    processed = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    processed = processed / 255.0  # Normalize
    image_sequence.append(processed)

# --- Simulation Loop ---
print("🚗 Starting simulated self-driving loop. Press 'q' to quit.")

while True:
    new_frame = capture_frame()
    processed_frame = cv2.resize(new_frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    processed_frame = processed_frame / 255.0

    image_sequence.append(processed_frame)
    if len(image_sequence) > SEQUENCE_LENGTH:
        image_sequence.pop(0)

    model_input = np.array([image_sequence])  # Shape: (1, SEQUENCE_LENGTH, HEIGHT, WIDTH, 3)

    try:
        prediction = model.predict(model_input, verbose=0)[0][0]
        print(f"Predicted Steering Angle: {prediction:.4f}")
    except Exception as e:
        print(f"Error during prediction: {e}")
        break

    # --- Display Frame ---
    cv2.imshow("Simulated Camera Feed", (processed_frame * 255).astype(np.uint8))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
print("✅ Simulation ended.")
